In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import logging
import requests
import datetime as dt
import pylab

import scipy.stats as stats
from matplotlib import pyplot

import numpy as np

import warnings
warnings.filterwarnings('ignore')

logging.getLogger().setLevel(logging.INFO)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os, sys

home = os.environ["HOME"]
sys.path.append(f"{home}/Desktop/project-ursa/lib")

import imp

In [39]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import logging
import requests
import datetime as dt
import pylab

import scipy.stats as stats
from matplotlib import pyplot

import numpy as np

import warnings
warnings.filterwarnings('ignore')

logging.getLogger().setLevel(logging.INFO)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os, sys

home = os.environ["HOME"]
sys.path.append(f"{home}/Desktop/project-ursa/lib")

import imp



import data_utils
import monte_carlo
import machine_npv
import implied_payback_time
import mining_constants
import delta_hedge
import machine_replication
import analysis

imp.reload(data_utils)
imp.reload(monte_carlo)
imp.reload(machine_npv)
imp.reload(mining_constants)
imp.reload(implied_payback_time)
imp.reload(delta_hedge)
imp.reload(machine_replication)
imp.reload(analysis)



##########################################################################################
## 
##########################################################################################
du = data_utils.DataUtility("")
du._raw_network_price_df    = pd.read_csv("../../data/raw_network_price.csv").drop("Unnamed: 0", axis="columns")
du._raw_network_price_df.time = pd.to_datetime(du._raw_network_price_df.time)
du._df_feature() 
du._aggregate_weekly_df() 

machine_prices = data_utils.machine_market_prices("../../data/machine_prices.csv")
du._weekly_machine_price_df = du._join_machine_price(machine_prices, HR_ROLLING_AVG=30)

weekly_df = du._weekly_network_price_df
machine_df = du._weekly_machine_price_df


##########################################################################################
## 
##########################################################################################
imp.reload(monte_carlo)
machine_params   = mining_constants.machine_params()
machine_duration = 1.2
ky = mining_constants.MACHINE_M20
params = {
    "asic_energy_consumption" : machine_params[ky]["energy_consumption"],
    "asic_hash_rate"          : machine_params[ky]["hash_rate"],
    "electricity_cost"        : 0.03,
    "machine_duration"        : machine_duration,
    "sample_rate"             : mining_constants.SAMPLE_RATE_WEEKLY,
    "mc_size"                 : 1000,
    "interest_rate"           : 0.05,
    "asic_number"             : 1,
    "asic_efficiency"         : 1.0,
    "pool_fee"                : 0.005,
    "analysis_denomination"   : mining_constants.ASSET_USD
}

sp = monte_carlo.GBMPriceLinearHashRate()
sp.fit(weekly_df, sample_rate = mining_constants.SAMPLE_RATE_WEEKLY)
m_usd_bsm = machine_npv.MachineNPV_USD_BSM(sp, params)

##########################################################################################
## 
##########################################################################################
imp.reload(implied_payback_time)
ipt    = implied_payback_time.ImpliedPayoffTime(params, m_usd_bsm, machine_df.tail(4).head(1))
ipt_df = ipt.compute(machine_type=mining_constants.MACHINE_M20, alpha=0.75, beta=0.5, MAX_DURATION=5)
start_time, hash_rate, btc_close, option_bundle_init, machine_cost, ipb_ttm  = ipt.extract_bundle(k=0)

##########################################################################################
## 
##########################################################################################
imp.reload(machine_replication)
replication_params = {
    "interest_rate" : params["interest_rate"], 
    "slippage" : 0.0
}

# Simulate RND GBM Prices.
sp._gbm_params["mu"] = 0# sp._gbm_params["mu"] = np.power(1 + params["interest_rate"], 1/params["sample_rate"]) - 1

ttm_vec, ttm_dt_vec, St, HRt, wt, Ht = sp.simulate(
    start_time    = start_time,
    spot_init     = btc_close,
    hashrate_init = hash_rate,
    sample_rate   = mining_constants.SAMPLE_RATE_HOURLY,
    N = 100,
    T = 2
)

res_lst = []
lambda_C = 1.0
for k in range(St.shape[1]):

    ## Initialize dataframe for all tracked metrics 
    price_df = pd.DataFrame({"ttm" : ttm_vec, "time": ttm_dt_vec, "price": St[:, k], "wt": wt}).set_index("time")
    hedge_iv = sp._gbm_params.get("sigma") * np.sqrt(mining_constants.WEEK_IN_YEAR)

    ## Initialize dataframe for all tracked metrics 
    mr = machine_replication.MachineReplication(option_bundle_init, price_df, replication_params)
    r  = mr.process()

    ## Initialize dataframe for all tracked metrics 
    tmo = machine_replication.TrueMiningOutput(mr, sp, params, option_bundle_init)
    tmo._generate_true_performance(start_time, hash_rate, lambda_C)
    result_df = tmo._generate_result_df()
    metrics   = tmo._generate_metrics(machine_cost)
    logging.info(metrics)
    res_lst.append(metrics)

df = pd.DataFrame(res_lst)

INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.019819298828056494, 'tracking_error_true': -0.019819298828056494, 'short_shortfall_init': 5.828670879282072e-16, 'short_shortfall_true': 5.828670879282072e-16, 'weekly_liq_unhedged_init': 3200.296015320487, 'weekly_liq_unhedged_true': 3200.296015320487, 'unhedged_tracking_error_init': 1.3705896409781386, 'unhedged_tracking_error_true': 1.3705896409781386}
INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': 0.18991970674785635, 'tracking_error_true': -0.015052169737402367, 'short_shortfall_init': 0.023779039810970626, 'short_shortfall_true': -1.249000902703301e-16, 'weekly_liq_unhedged_init': 803.0578699877932, 'weekly_liq_unhedged_true': 803.0578699877932, 'unhedged_tracking_error_init': -0.4051423185275606, 'unhedged_tracking_error_true': -0.405142318527

INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.018104623620698385, 'tracking_error_true': -0.018104623620698385, 'short_shortfall_init': -2.7755575615628914e-16, 'short_shortfall_true': -2.7755575615628914e-16, 'weekly_liq_unhedged_init': 1344.7493382676653, 'weekly_liq_unhedged_true': 1344.7493382676653, 'unhedged_tracking_error_init': -0.0038893790609886817, 'unhedged_tracking_error_true': -0.0038893790609886817}
INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.014510046721868782, 'tracking_error_true': -0.014510046721868782, 'short_shortfall_init': 6.106226635438361e-16, 'short_shortfall_true': 6.106226635438361e-16, 'weekly_liq_unhedged_init': 750.8992651659182, 'weekly_liq_unhedged_true': 750.8992651659182, 'unhedged_tracking_error_init': -0.44377832209931983, 'unhedged_tracking_error_true'

INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.017360423919008872, 'tracking_error_true': -0.017360423919008872, 'short_shortfall_init': 4.718447854656915e-16, 'short_shortfall_true': 4.718447854656915e-16, 'weekly_liq_unhedged_init': 1480.3095614362958, 'weekly_liq_unhedged_true': 1480.3095614362958, 'unhedged_tracking_error_init': 0.0965256010639228, 'unhedged_tracking_error_true': 0.0965256010639228}
INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.019926171044737623, 'tracking_error_true': -0.019926171044737623, 'short_shortfall_init': -5.551115123125783e-16, 'short_shortfall_true': -5.551115123125783e-16, 'weekly_liq_unhedged_init': 2579.5485872231366, 'weekly_liq_unhedged_true': 2579.5485872231366, 'unhedged_tracking_error_init': 0.9107767312763975, 'unhedged_tracking_error_true': 0.910776

INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': 0.11628853542642048, 'tracking_error_true': -0.0091414032493482, 'short_shortfall_init': 0.06003107356087592, 'short_shortfall_true': -4.3021142204224816e-16, 'weekly_liq_unhedged_init': 483.6696303329619, 'weekly_liq_unhedged_true': 483.6696303329619, 'unhedged_tracking_error_init': -0.6417261997533615, 'unhedged_tracking_error_true': -0.6417261997533615}
INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': 0.026478792793615407, 'tracking_error_true': -0.016566726369130286, 'short_shortfall_init': 0.0076692765920553935, 'short_shortfall_true': -2.7755575615628914e-17, 'weekly_liq_unhedged_init': 893.0415344376358, 'weekly_liq_unhedged_true': 893.0415344376358, 'unhedged_tracking_error_init': -0.3384877522684179, 'unhedged_tracking_error_true': -0.33848775226

INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.01901176137529655, 'tracking_error_true': -0.01901176137529655, 'short_shortfall_init': 2.7755575615628914e-17, 'short_shortfall_true': 2.7755575615628914e-17, 'weekly_liq_unhedged_init': 1807.5038878714029, 'weekly_liq_unhedged_true': 1807.5038878714029, 'unhedged_tracking_error_init': 0.33889176879363175, 'unhedged_tracking_error_true': 0.33889176879363175}
INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': 0.15339813022883092, 'tracking_error_true': -0.009884739161721015, 'short_shortfall_init': 0.04637192079918247, 'short_shortfall_true': 8.326672684688674e-17, 'weekly_liq_unhedged_init': 366.6766057009379, 'weekly_liq_unhedged_true': 366.6766057009379, 'unhedged_tracking_error_init': -0.7283876994807867, 'unhedged_tracking_error_true': -0.7283876994

INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.01916788982316872, 'tracking_error_true': -0.01916788982316872, 'short_shortfall_init': -7.494005416219807e-16, 'short_shortfall_true': -7.494005416219807e-16, 'weekly_liq_unhedged_init': 1738.8745902054316, 'weekly_liq_unhedged_true': 1738.8745902054316, 'unhedged_tracking_error_init': 0.2880552520040234, 'unhedged_tracking_error_true': 0.2880552520040234}
INFO:root:simulate: daily_reward x (365 / 52) = daily_reward x (7.019230769230769)
INFO:root:{'machine_cost': 1350.0, 'tracking_error_ideal': -0.018890099330197172, 'tracking_error_true': -0.018890099330197172, 'short_shortfall_init': -1.1102230246251565e-16, 'short_shortfall_true': -1.1102230246251565e-16, 'weekly_liq_unhedged_init': 2152.9547964852422, 'weekly_liq_unhedged_true': 2152.9547964852422, 'unhedged_tracking_error_init': 0.5947813307298091, 'unhedged_tracking_error_true': 0.5947

In [40]:
df.head()

,machine_cost,tracking_error_ideal,tracking_error_true,short_shortfall_init,short_shortfall_true,weekly_liq_unhedged_init,weekly_liq_unhedged_true,unhedged_tracking_error_init,unhedged_tracking_error_true
0,1350.0,-0.019819,-0.019819,5.828671e-16,5.828671e-16,3200.296015,3200.296015,1.370590,1.370590
1,1350.0,0.189920,-0.015052,2.377904e-02,-1.249001e-16,803.057870,803.057870,-0.405142,-0.405142
2,1350.0,-0.018701,-0.018701,1.332268e-15,1.332268e-15,1999.928957,1999.928957,0.481429,0.481429
3,1350.0,-0.019921,-0.019921,-4.718448e-16,-4.718448e-16,7588.703080,7588.703080,4.621262,4.621262
4,1350.0,-0.017109,-0.017109,3.608225e-16,3.608225e-16,1418.797276,1418.797276,0.050961,0.050961


In [41]:
df.unhedged_tracking_error_init.quantile(0.05)

-0.7766568832379839

In [42]:
df.unhedged_tracking_error_init.mean()

0.10647265237705868

In [43]:
df.unhedged_tracking_error_init.std()

0.910855350565723

In [65]:



result_df = pd.read_csv("../../data/lambda_sensitivity_results_3.csv")

print(result_df.groupby("lambda_C")[["tracking_error_true"]].mean())
print(result_df.groupby("lambda_C")[["tracking_error_true"]].std())
print(result_df.groupby("lambda_C")[["tracking_error_true"]].quantile(0.05))

          tracking_error_true
lambda_C                     
0.8                  0.037680
0.9                  0.010254
1.0                 -0.015369
1.1                 -0.041460
1.2                 -0.064959
          tracking_error_true
lambda_C                     
0.8                  0.033140
0.9                  0.017868
1.0                  0.004061
1.1                  0.018995
1.2                  0.033164
          tracking_error_true
lambda_C                     
0.8                 -0.002703
0.9                 -0.009214
1.0                 -0.019901
1.1                 -0.075869
1.2                 -0.128953


In [66]:
print(result_df.groupby("lambda_C")[["unhedged_tracking_error_true"]].mean())
print(result_df.groupby("lambda_C")[["unhedged_tracking_error_true"]].std())
print(result_df.groupby("lambda_C")[["unhedged_tracking_error_true"]].quantile(0.05))

          unhedged_tracking_error_true
lambda_C                              
0.8                           0.016038
0.9                           0.003649
1.0                          -0.008345
1.1                          -0.020014
1.2                          -0.031371
          unhedged_tracking_error_true
lambda_C                              
0.8                           0.687000
0.9                           0.676419
1.0                           0.666208
1.1                           0.656384
1.2                           0.646933
          unhedged_tracking_error_true
lambda_C                              
0.8                          -0.782236
0.9                          -0.783170
1.0                          -0.784090
1.1                          -0.784998
1.2                          -0.786135


In [ ]:

# offset = np.mean(result_df[result_df.lambda_C == 1.0].tracking_error_true.values)
offset = 0
data_08 = result_df[result_df.lambda_C == 0.8].tracking_error_true.values - offset
data_09 = result_df[result_df.lambda_C == 0.9].tracking_error_true.values - offset
data_10 = result_df[result_df.lambda_C == 1.0].tracking_error_true.values - offset
data_11 = result_df[result_df.lambda_C == 1.1].tracking_error_true.values - offset
data_12 = result_df[result_df.lambda_C == 1.2].tracking_error_true.values - offset
data    = [data_08, data_09, data_10, data_11, data_12]

var95   = [np.quantile(data_08, 0.05), np.quantile(data_09, 0.05), 
           np.quantile(data_10, 0.05), np.quantile(data_11, 0.05), np.quantile(data_12, 0.05)] 
    
    
fig = plt.figure(figsize =(5, 4))
bp = plt.boxplot(data)
plt.plot([1, 2, 3, 4, 5], var95)
plt.xticks([1, 2, 3, 4, 5], ["0.8", "0.9", "1.0", "1.1", "1.2"])
plt.ylabel("Tracking Error")
plt.xlabel("$\lambda_{TR}/\lambda_{I}$")
plt.grid()
plt.show()



# np.mean(result_df[result_df.lambda_C == 1.0].unhedged_tracking_error_true.values)

# result_df["unhedged_tracking_error_init"] = (result_df["machine_cost"] - result_df["weekly_liq_unhedged_init"])/result_df["machine_cost"]
# result_df["unhedged_tracking_error_true"] = (result_df["machine_cost"] - result_df["weekly_liq_unhedged_true"])/result_df["machine_cost"]

# offset = np.mean(result_df[result_df.lambda_C == 1.0].unhedged_tracking_error_true.values)
# print(offset)

# # offset = 0
# data_08 = result_df[result_df.lambda_C == 0.8].unhedged_tracking_error_true.values - offset
# data_09 = result_df[result_df.lambda_C == 0.9].unhedged_tracking_error_true.values - offset
# data_10 = result_df[result_df.lambda_C == 1.0].unhedged_tracking_error_true.values - offset
# data_11 = result_df[result_df.lambda_C == 1.1].unhedged_tracking_error_true.values - offset
# data_12 = result_df[result_df.lambda_C == 1.2].unhedged_tracking_error_true.values - offset
# data    = [data_08, data_09, data_10, data_11, data_12]
 
# var95   = [np.quantile(data_08, 0.05), np.quantile(data_09, 0.05), 
#            np.quantile(data_10, 0.05), np.quantile(data_11, 0.05), np.quantile(data_12, 0.05)] 
    
# fig = plt.figure(figsize =(5, 4))
# bp = plt.boxplot(data)
# plt.plot([1, 2, 3, 4, 5], var95)
# plt.xticks([1, 2, 3, 4, 5], ["0.8", "0.9", "1.0", "1.1", "1.2"])
# plt.ylabel("Tracking Error")
# plt.xlabel("$\lambda_{TR}/\lambda_{I}$")
# plt.grid()
# plt.show()
# print(var95)